In [1]:
import numpy as np
import pandas as pd
import string
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
data = open("train.txt","r")
data=list(data)

test_data=open("test.txt","r")
test_data=list(test_data)

# Preprocessing

In [0]:
def preprocess(data):
  tweets=[]
  sentiments=[]
  arr=[]
  c=0
  for i in data:
    if(i=="\n"):
      tweets.append(arr)
      arr=[]
    else:
      temp=i.split("\t")
      if(len(temp)==3):
        sentiments.append(temp[2])
      arr.append(temp[0])
  tweet=[]
  for i in tweets:
    temp=""
    for j in i:
      temp+=j
      temp+=" "
    tweet.append(temp[:-1])
  tweets=tweet 
  sent=[]
  for i in sentiments:
    temp=i[:-1]
    if(temp=="neutral"):
      sent.append(0)
    elif(temp=="positive"):
      sent.append(1)
    else:
      sent.append(2)  

  return(tweets,sent)

In [0]:
 tweets,sent=preprocess(data)

In [15]:
len(tweets),len(sent)

(15131, 15131)

In [0]:
test_tweets,test_sent=preprocess(test_data)

# Model1 Bert

In [5]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [6]:
!pip install pytorch-pretrained-bert


In [7]:

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

Using TensorFlow backend.


In [8]:
sentences = tweets

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
labels = sent


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])


MAX_LEN = 178

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)


train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)


train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)



# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
100%|██████████| 995526/995526 [00:00<00:00, 5566331.02B/s]


Tokenize the first sentence:
['[CLS]', 'meta', '@', 'Adi', '##l', '##N', '##isar', '##B', '##utt', 'pak', '##istan', 'ka', 'gh', '##ra', 'tau', '##q', 'he', 'Pakistan', 'Israel', 'ko', 'tas', '##lee', '##m', 'nahe', '##in', 'ker', '##ta', 'Is', '##ko', 'Palestine', 'ke', '##hta', 'he', '-', 'O', '##CC', '##UP', '##IE', '##D', 'PAL', '##ES', '##TI', '##NE', '[SEP]']


In [9]:
model1 = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=3)
model1.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
  

In [10]:
param_optimizer = list(model1.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

t_total value of -1 results in schedule not being applied


In [0]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla K80'

In [13]:
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4


# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model1.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    loss = model1(b_input_ids,attention_mask=b_input_mask.long(), labels=b_labels)
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model1.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model1(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))



Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Train loss: 0.9802032275378983


Epoch:  25%|██▌       | 1/4 [15:24<46:13, 924.41s/it]

Validation Accuracy: 0.517578125
Train loss: 0.8359185822413001


Epoch:  50%|█████     | 2/4 [30:49<30:49, 924.72s/it]

Validation Accuracy: 0.6052083333333333
Train loss: 0.734068137869029


Epoch:  75%|███████▌  | 3/4 [46:17<15:25, 925.46s/it]

Validation Accuracy: 0.590234375
Train loss: 0.6176244839676109


Epoch: 100%|██████████| 4/4 [1:01:44<00:00, 926.06s/it]

Validation Accuracy: 0.5994791666666667


In [0]:
torch.save(model1, "bert.bin")

In [0]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

In [0]:
#bert test

test_tweets,test_sent=preprocess(test_data)

In [16]:
len(test_tweets),len(test_sent)

(1869, 1869)

In [0]:
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

def runBertCount(sentence):
    model=model1
    # model=torch.load('bert.bin')
    sentences = "[CLS] " + sentence +" [SEP]"
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)
    tokenized_texts = tokenizer.tokenize(sentences)
#     print(tokenized_texts)

    MAX_LEN = 178

    # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
    my_input_ids = [tokenizer.convert_tokens_to_ids(tokenized_texts)]
    # # Pad our input tokens
    my_input_ids = pad_sequences(my_input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    # # Create attention masks
    my_attention_masks = []
    # print(my_input_ids)
    # # Create a mask of 1s for each token followed by 0s for padding
#     print(my_input_ids)
    for seq in my_input_ids:
        seq_mask = [float(i>0) for i in seq]
        my_attention_masks.append(seq_mask)

    prediction_inputs = torch.tensor(my_input_ids)
    prediction_masks = torch.tensor(my_attention_masks)


    with torch.no_grad():
        prediction_inputs=prediction_inputs.to(device)
        prediction_masks=prediction_masks.to(device)
        logits = model(prediction_inputs.long(), attention_mask=prediction_masks.long())
        logits = logits.detach().cpu().numpy()
    return np.argmax(logits)

In [0]:
def accuracy(label,true_label):
  if(label==true_label):
    return(1)
  else:
    return(0)

In [0]:
y_true=[]
y_pred=[]

In [0]:
c=0
for i in range(len(test_tweets)):
  # print(runBertCount(test_tweets[i]))
  temp=runBertCount(test_tweets[i])
  c+=accuracy(temp,test_sent[i])
  y_true.append(test_sent[i])
  y_pred.append(temp)
  print(c/(i+1))

print(c/len(test_tweets))
  

In [24]:
from sklearn.metrics import classification_report
target_names = ['neutral', 'positive', 'negative']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     neutral       0.53      0.55      0.54       754
    positive       0.60      0.56      0.58       582
    negative       0.57      0.58      0.58       533

    accuracy                           0.56      1869
   macro avg       0.56      0.56      0.56      1869
weighted avg       0.56      0.56      0.56      1869

